In [1]:
using StatsBase
using Plots
using LaTeXStrings

In [2]:
function gaussian(μ, σ, arr)
   
    return [exp(-(item-μ)^2/(2*σ^2)) / sqrt(2*pi*σ^2) for item in arr]
    
end

function init_layers(x, y, S0, P0, method_P="rand", method_S="rand", σ=1.5)

    if method_S == "rand"
        
        if S0 == x*y
            
            S_pinna = [i for i in 1 : x*y]
            
        else
    
            S_pinna = sample(1:x*y, S0, replace=false) #Indices where S pinna are
            
        end
        
    elseif method_S == "custom"
       
        S_pinna = findall(vec(S0) .> 0)
        
    end

    if method_P == "rand"
    
        #Flattened array of P. Each index is a cell and its value the number of P in that cell
        P = zeros(x*y)
        P[rand(1:x*y)] = P0  
        
    elseif method_P == "center"
        
        P = zeros(x, y)
        
        P[Int32(floor(x/2)), Int32(floor(y/2))] = P0
        
        P = vec(P)
        
    elseif method_P == "gaussian"
       
        P = zeros(x,y)
        
        Ycells = 1 : 1 : y

        P_i = Int32.(round.(gaussian(mean(Ycells), σ, Ycells) * P0))
        
        for i in 1 : x
    
            P[:, i] = P_i
    
        end
        
        P = vec(P)
        
    elseif method_P == "custom"
        
        P = vec(P0)
        
    end
    
    #This could change if we want to implement initial infected individuals
    I_pinna = [] #Indices where I pinna are
    R_pinna = [] #Indices where R pinna are
    
   
    return S_pinna, I_pinna, R_pinna, P 
    
end

function compute_waiting_time(S_pos, I_pos, P, parameters)
    
    #Here I am not considering parasite mobility as a process which may compete with all the other
    #epidemiological processes. But I've seen a paper where parasite mobility is considered as another process.
   
    γ, λ, μ, β, κ = parameters
    
    γ_tot = length(I_pos) * γ #N_S * γ
    λ_tot = length(I_pos) * λ #N_I * λ
    
    μ_tot = sum(P) * μ #N_P * μ
    
    if length(P[S_pos]) != 0
    
        β_tot = sum(P[S_pos]) * β #\sum_{i=1}^N_S β*P_{cell}=β\sum_{i=1}^N_S P_{cell}
        
    else
        
        β_tot = 0.0
        
    end
        
    κ_tot = sum(P) * κ
    
    W = γ_tot + λ_tot + μ_tot + β_tot + κ_tot
    
    tau = -1/W * log(rand())
    
    return tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W
    
end

function compute_waiting_time_MF(S_pos, I_pos, P, parameters)
    
    #Here I am not considering parasite mobility as a process which may compete with all the other
    #epidemiological processes. But I've seen a paper where parasite mobility is considered as another process.
   
    γ, λ, μ, β, κ = parameters
    
    γ_tot = length(I_pos) * γ #N_S * γ
    λ_tot = length(I_pos) * λ #N_I * λ
    
    μ_tot = sum(P) * μ #N_P * μ
    
    if length(P) != 0
    
        β_tot = sum(P) * length(S_pos) *  β #\sum_{i=1}^N_S β*P_{cell}=β\sum_{i=1}^N_S P_{cell}
        
    else
        
        β_tot = 0.0
        
    end
        
    κ_tot = sum(P) * κ
    
    W = γ_tot + λ_tot + μ_tot + β_tot + κ_tot
    
    tau = -1/W * log(rand())
    
    return tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W
    
end

function choose_apply_event(total_rates, parameters, positions, x, y, W)
    
    #Here I am not considering parasite mobility as a process which may compete with all the other
    #epidemiological processes. But I've seen a paper where parasite mobility is considered as another process.
    
    γ_tot, λ_tot, μ_tot, β_tot, κ_tot = total_rates
    
    γ, λ, μ, β, κ = parameters
    
    S_pos, I_pos, R_pos, P = positions
    
    U = rand() * W
    
    if U < κ_tot #Parasite mobility
        
        P = parasite_mobility_RW(x, y, P)
        
    elseif U < (κ_tot + μ_tot)#A parasite dies
        
        #All parasites have same prob of dying, choose one at random
        P[rand(findall(P .> 0))] -= 1
        
    elseif U < (κ_tot + μ_tot + β_tot)  #A susceptible pinna gets infected
        
        weights = P[S_pos] .* (β/β_tot) #Array where each item is the number of P for each Pina cell
        S_ind = [i for i in 1 : length(S_pos)]
        
        idx = sample(S_ind, Weights(weights)) #Pinna that gets infected
        
        P[S_pos[idx]] -= 1
        
        append!(I_pos, S_pos[idx]) #Append the position of the S individual in I list

        splice!(S_pos, idx) #Delete the position of S list
    
    elseif U < (γ_tot + κ_tot + μ_tot + β_tot) #An infected pinna individual dies
        
        #All infected pinna have same prob of dying, choose one at random
        idx = rand(1 : length(I_pos))
        
        append!(R_pos, I_pos[idx])
        
        splice!(I_pos, idx)
        
    else #An infected pinna individual produces parasite
  
        #All infected pinna have same proba of producing parasite, choose one at random

        P[rand(I_pos)] += 1 #Append the created parasite in the position s
                
    end
    
    return S_pos, I_pos, R_pos, P
    
end 

function choose_apply_event_MF(total_rates, parameters, positions, x, y, W)
    
    #Here I am not considering parasite mobility as a process which may compete with all the other
    #epidemiological processes. But I've seen a paper where parasite mobility is considered as another process.
    
    γ_tot, λ_tot, μ_tot, β_tot, κ_tot = total_rates
    
    γ, λ, μ, β, κ = parameters
    
    S_pos, I_pos, R_pos, P = positions
    
    U = rand() * W
    
    if U < γ_tot #An infected pinna individual dies
        
        #All infected pinna have same prob of dying, choose one at random
        idx = rand(1 : length(I_pos))
        
        append!(R_pos, I_pos[idx])
        
        splice!(I_pos, idx)
        
    elseif U < (γ_tot + λ_tot) #An infected pinna individual produces parasite
  
        #All infected pinna have same proba of producing parasite, choose one at random

        P[rand(I_pos)] += 1 #Append the created parasite in the position s
                
    elseif U < (γ_tot + λ_tot + μ_tot)#A parasite dies
        
        #All parasites have same prob of dying, choose one at random
        P[rand(findall(P .> 0))] -= 1
         
    elseif U < (γ_tot + λ_tot + μ_tot + β_tot)  #A susceptible pinna gets infected
        
        S_ind = [i for i in 1 : length(S_pos)]
        
        idx = rand(S_ind) #Pinna that gets infected
        
        P[S_pos[idx]] -= 1
        
        append!(I_pos, S_pos[idx]) #Append the position of the S individual in I list

        splice!(S_pos, idx) #Delete the position of S list
        
    else #Parasite mobility
        
        P = parasite_mobility_RW(x, y, P)
        
    end
    
    return S_pos, I_pos, R_pos, P
    
end 

#Microscopic RW
function parasite_mobility_RW(x, y, P)
    
    valid_Ps_idx = findall(P .> 0)
    valid_Ps_N = P[valid_Ps_idx]
    
    idx = sample(valid_Ps_idx, Weights(Float64.(valid_Ps_N)))
    
    #Choice movement
    opt = rand([1, 2, 3, 4])
    
    if opt == 1 #up
    
        new_idx = mod1(idx + x, x*y)
        
    elseif opt == 2 #down
        
        new_idx = mod1(idx - x, x*y)
        
    elseif opt == 3 #right
    
        if idx % x == 0

            new_idx = idx - x + 1

        else

            new_idx = idx + 1

        end
        
    else #left
    
        if (idx - 1) % x == 0

            new_idx = idx + x - 1

        else

            new_idx = idx - 1

        end
        
    end
    
    P[idx] -= 1
    P[new_idx] += 1
    
    return P
    
end

function get_xy_positions(M, flat_arr)
    
    Cartesian_ind = CartesianIndices(M)[flat_arr]

    y = [idx[1] for idx in Cartesian_ind]
    x = [idx[2] for idx in Cartesian_ind]
    
    return x, y
    
end

function IBM(t_end, x, y, S0, P0, parameters, method_P, method_S)
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    S_t = [length(S_pos)]
    I_t = [length(I_pos)]
    R_t = [length(R_pos)]
    P_t = [sum(P)]
    
    time = [0.0]
    
    t = 0
    
    while t < t_end
    
        #println(t, " ", P)
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time(S_pos, I_pos, P, parameters)
        
        if tau < 0 || (length(I_pos) == 0 && sum(P) == 0)
            
            break
            
        end

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]
        
        S_pos, I_pos, R_pos, P = choose_apply_event(total_rates, parameters, positions, x, y, W)
        
        t += tau
        
        #Record values of interest
        append!(S_t, length(S_pos))
        append!(I_t, length(I_pos))
        append!(R_t, length(R_pos))
        append!(P_t, sum(P))        
        
        append!(time, t)
        
    end
    
    return S_t, I_t, R_t, P_t, time
    
end

function IBM_fast(t_end, x, y, S0, P0, parameters, method_P, method_S; filename="record.txt")
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    t = 0
    
    f = open(filename, "w")
    
    println(f, "#t\tS\tI\tR\tP")
    
    println(f, t, "\t", length(S_pos), "\t", length(I_pos), "\t", length(R_pos), "\t", sum(P))
    
    while t < t_end
    
        #println(t, " ", P)
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time(S_pos, I_pos, P, parameters)
        
        if tau < 0 || (length(I_pos) == 0 && sum(P) == 0)
            
            break
            
        end

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]

        S_pos, I_pos, R_pos, P = choose_apply_event(total_rates, parameters, positions, x, y, W)
        
        t += tau
        
        println(f, t, "\t", length(S_pos), "\t", length(I_pos), "\t", length(R_pos), "\t", sum(P))
        
    end
    
    close(f)
    
    return S_pos, I_pos, R_pos, P #Return last config
    
end

function IBM_final_state(x, y, S0, P0, parameters, method_P, method_S)
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    while (length(I_pos) > 0 || sum(P) > 0)
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time(S_pos, I_pos, P, parameters)

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]

        S_pos, I_pos, R_pos, P = choose_apply_event(total_rates, parameters, positions, x, y, W)
        
    end
    
    return length(S_pos), length(R_pos)
    
end

function IBM_MF(t_end, x, y, S0, P0, parameters, method_P, method_S)
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    S_t = [length(S_pos)]
    I_t = [length(I_pos)]
    R_t = [length(R_pos)]
    P_t = [sum(P)]
    
    time = [0.0]

    t = 0
    
    while t < t_end
    
        #println(t, " ", P)
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time_MF(S_pos, I_pos, P, parameters)
        
        if tau < 0 || (length(I_pos) == 0 && sum(P) == 0)
            
            break
            
        end

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]

        S_pos, I_pos, R_pos, P = choose_apply_event_MF(total_rates, parameters, positions, x, y, W)

        #println(findall(P .< 0))
        
        t += tau
        
        #Record values of interest
        append!(S_t, length(S_pos))
        append!(I_t, length(I_pos))
        append!(R_t, length(R_pos))
        append!(P_t, sum(P))        
        
        append!(time, t)
        
    end
    
    return S_t, I_t, R_t, P_t, time
    
end


function IBM_animation(t_end, x, y, S0, P0, parameters, method_P, method_S; every_frame=1, ms=8, clims=(0, 100))
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    S_t = [length(S_pos)]
    I_t = [length(I_pos)]
    R_t = [length(R_pos)]
    P_t = [sum(P)]
    
    time = [0.0]

    t = 0
    
    #Initialize plots
    M = reshape(P, (x,y))
    
    X_S, Y_S = get_xy_positions(M, S_pos)
    X_I, Y_I = get_xy_positions(M, I_pos)
    X_R, Y_R = get_xy_positions(M, R_pos)

    N_P = sum(P)

    N_S = length(S_pos)
    N_I = length(I_pos)
    N_R = length(R_pos)

    p1 = heatmap(M, c=:blues, clims=clims)
    
    scatter!(X_S, Y_S, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:green3, m=:v, ms=ms, label="")
    scatter!(X_I, Y_I, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:red, m=:v, ms=ms, label="")
    scatter!(X_R, Y_R, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:black, m=:v, ms=ms, label="")
    
    p2 = plot(time, P_t, label="P", color=:deepskyblue)
    
    p3 = plot(time, [S_t I_t R_t], color=[:green3 :red :black], labels=["S" "I" "R"], legend=:topleft)
    
    plot(p1, p2, p3, layout=@layout[a [b ; c]], size=(1200, 500))
    
    t = 0
    
    anim = @animate for i in 1 : 1000000 
    
        if t >= t_end
        
            break
        
        end
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time(S_pos, I_pos, P, parameters)
        
        if tau < 0 || (length(I_pos) == 0 && sum(P) == 0)
            
            break
            
        end

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]

        S_pos, I_pos, R_pos, P = choose_apply_event(total_rates, parameters, positions, x, y, W)

        t += tau
        
        #Plot
        M = reshape(P, (x,y))
            
        X_S, Y_S = get_xy_positions(M, S_pos)
        X_I, Y_I = get_xy_positions(M, I_pos)
        X_R, Y_R = get_xy_positions(M, R_pos)

        N_P = sum(P)

        N_S = length(S_pos)
        N_I = length(I_pos)
        N_R = length(R_pos)
        
        append!(S_t, N_S)
        append!(I_t, N_I)
        append!(R_t, N_R)
        append!(P_t, N_P)
        
        append!(time, t)
        
        p1 = heatmap(M, c=:blues, clims=clims)
        
        #=
        for i in 1 : size(M)[1]
    
            for j in 1 : size(M)[2]
                
                if j ∉ X_S && i ∉ Y_S

                    annotate!(j, i, text(M[i,j], 8))
                    
                end

            end

        end
        =#
        
        scatter!(X_S, Y_S, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:green3, m=:v, ms=ms, label="")
        scatter!(X_I, Y_I, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:red, m=:v, ms=ms, label="")
        scatter!(X_R, Y_R, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:black, m=:v, ms=ms, label="")
        
        p2 = plot(time, P_t, color=:deepskyblue, lw=3, label="P", legend=:none)
        
        p3 = plot(time, [S_t I_t R_t], color=[:green3 :red :black], lw=3, legend=:none)
        
        plot(p1, p2, p3, layout=@layout[a [b ; c]], size=(1200, 500))
    
        t += tau
        
    end every every_frame
    
    return anim
    
end

function IBM_animation_fast(t_end, x, y, S0, P0, parameters, method_P, method_S; every_frame=1, ms=8, clims=(0, 100))
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    t = 0
    
    #Initialize plots
    M = reshape(P, (x,y))
    
    X_S, Y_S = get_xy_positions(M, S_pos)
    X_I, Y_I = get_xy_positions(M, I_pos)
    X_R, Y_R = get_xy_positions(M, R_pos)

    N_P = sum(P)

    N_S = length(S_pos)
    N_I = length(I_pos)
    N_R = length(R_pos)

    p1 = heatmap(M, c=:blues, clims=clims)
    
    scatter!(X_S, Y_S, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:green3, m=:v, ms=ms, label="")
    scatter!(X_I, Y_I, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:red, m=:v, ms=ms, label="")
    scatter!(X_R, Y_R, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:black, m=:v, ms=ms, label="")
    
    plot(p1, size=(1000, 800))
    
    t = 0
    
    anim = @animate for i in 1 : 1000000 
    
        if t >= t_end
        
            break
        
        end
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time(S_pos, I_pos, P, parameters)
        
        if tau < 0 || (length(I_pos) == 0 && sum(P) == 0)
            
            break
            
        end

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]

        S_pos, I_pos, R_pos, P = choose_apply_event(total_rates, parameters, positions, x, y, W)

        t += tau
        
        #Plot
        M = reshape(P, (x,y))
            
        X_S, Y_S = get_xy_positions(M, S_pos)
        X_I, Y_I = get_xy_positions(M, I_pos)
        X_R, Y_R = get_xy_positions(M, R_pos)

        
        p1 = heatmap(M, c=:blues, clims=clims)
        
        scatter!(X_S, Y_S, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:green3, m=:v, ms=ms, label="")
        scatter!(X_I, Y_I, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:red, m=:v, ms=ms, label="")
        scatter!(X_R, Y_R, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:black, m=:v, ms=ms, label="")
        
        plot(p1, size=(1000, 800))
    
        t += tau
        
    end every every_frame
    
    return anim
    
end

function IBM_gif(t_end, x, y, S0, P0, parameters, method_P, method_S ; every_frame, ms=8, size=(1000, 800), clims=(0, 100))
    
    γ, λ, μ, β, κ = parameters
    
    #Initialize variables
    S_pos, I_pos, R_pos, P = init_layers(x, y, S0, P0, method_P, method_S)
    
    S_t = [length(S_pos)]
    I_t = [length(I_pos)]
    R_t = [length(R_pos)]
    P_t = [sum(P)]
    
    time = [0.0]

    t = 0
    
    #Initialize plots
    M = reshape(P, (x,y))
    
    X_S, Y_S = get_xy_positions(M, S_pos)
    X_I, Y_I = get_xy_positions(M, I_pos)
    X_R, Y_R = get_xy_positions(M, R_pos)

    N_P = sum(P)

    N_S = length(S_pos)
    N_I = length(I_pos)
    N_R = length(R_pos)

    p1 = heatmap(M, c=:blues, clims=clims, size=size)
    
    scatter!(X_S, Y_S, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:green3, m=:v, ms=ms, label="")
    scatter!(X_I, Y_I, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:red, m=:v, ms=ms, label="")
    scatter!(X_R, Y_R, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:black, m=:v, ms=ms, label="")
    
    p2 = plot(time, P_t, label="P", color=:deepskyblue)
    
    p3 = plot(time, [S_t I_t R_t], color=[:green3 :red :black], labels=["S" "I" "R"], legend=:topleft)
    
    plot(p1, p2, p3, layout=@layout[a [b ; c]], size=(1000, 500))
    
    t = 0
    
    @gif for i in 1 : 100000 
    
        if t >= t_end
        
            break
        
        end
        
        #Compute total rate
        tau, γ_tot, λ_tot, μ_tot, β_tot, κ_tot, W = compute_waiting_time(S_pos, I_pos, P, parameters)
        
        if tau < 0 || (length(I_pos) == 0 && sum(P) == 0)
            
            break
            
        end

        #Choose event to happen
        total_rates = [γ_tot, λ_tot, μ_tot, β_tot, κ_tot]
        positions = [S_pos, I_pos, R_pos, P]

        S_pos, I_pos, R_pos, P = choose_apply_event(total_rates, parameters, positions, x, y, W)

        t += tau
        
        #Plot
        M = reshape(P, (x,y))
            
        X_S, Y_S = get_xy_positions(M, S_pos)
        X_I, Y_I = get_xy_positions(M, I_pos)
        X_R, Y_R = get_xy_positions(M, R_pos)

        N_P = sum(P)

        N_S = length(S_pos)
        N_I = length(I_pos)
        N_R = length(R_pos)
        
        append!(S_t, N_S)
        append!(I_t, N_I)
        append!(R_t, N_R)
        append!(P_t, N_P)
        
        append!(time, t)
        
        p1 = heatmap(M, c=:blues, clims=clims, size=size)
        
        for i in 1 : size(M)[1]
    
            for j in 1 : size(M)[2]
                
                if j ∉ X_S && i ∉ Y_S

                    annotate!(j, i, text(M[i,j], 8))
                    
                end

            end

        end
        
        scatter!(X_S, Y_S, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:green3, m=:v, ms=ms, label="")
        scatter!(X_I, Y_I, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:red, m=:v, ms=ms, label="")
        scatter!(X_R, Y_R, xlim=(0.5, x+0.5), ylim=(0.5, y+0.5), color=:black, m=:v, ms=ms, label="")
        
        p2 = plot(time, P_t, color=:deepskyblue, lw=3, label="P", legend=:none)
        
        p3 = plot(time, [S_t I_t R_t], color=[:green3 :red :black], lw=3, legend=:none)
        
        plot(p1, p2, p3, layout=@layout[a [b ; c]], size=(1000, 500))
    
        t += tau
        
    end every every_frame
    
end

IBM_gif (generic function with 1 method)

In [19]:
x = 50
y = 50

S0 = x*y

P0 = 0.5 * x*y

parameters = [1.0, 50, 1, 1, 1.0]; # γ, λ, μ, β, κ

method_P = "center"

method_S = "rand"

"rand"

In [20]:
fs = @time IBM_final_state(x, y, S0, P0, parameters, method_P, method_S)

println(fs[1] / (x*y))

 68.246812 seconds (1.77 G allocations: 30.932 GiB, 9.29% gc time)
0.6164
